## After GPR-Lidar Fusion

In [ ]:
import datetime
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt

## Identify the multiple GPR points within LiDAR cells and find median TWTs
 - be sure to update CSV name
 - remoce random IX and lidar SD column when proper data updated
 - update column names 

In [ ]:
#import data change name based on csv file
dataFusion = pd.read_csv('GPR_ALL.csv')

dataFusion = geopandas.GeoDataFrame(dataFusion, geometry = geopandas.points_from_xy(dataFusion.easting, dataFusion.northing))
# Make lidar X,Y the geom column 

# %whos - this will tell you what variables you have 

#filter by date to seperate 2 data sets
date1 = '2020-02-02'
date2 = '2020-02-04'
dataFusion['date'] = pd.to_datetime(dataFusion['date'])
GPR1 = dataFusion[(dataFusion['date'] <= date1)] #less than and equal to
GPR2 = dataFusion[(dataFusion['date'] >= date2)]  #greater than and equal to this date 
print(len(GPR1), ',', len(GPR2))

### THIS IS TEMPORARYYY!! Delete when we have real data
##add fake columns for GPR to test out lidar 
#will delete when have real data

lidarSD = np.ones((len(GPR1),1))
GPR1['lidarSD'] = lidarSD
GPR1.head()

test1 = range(500000)
test1 = np.random.choice(test1, size=len(GPR1))
GPR1['test'] = test1

test2 = range(50000)
test2 = np.random.choice(test2, size = len(GPR2))
GPR2['test'] = test2

lidarSD = np.ones((len(GPR2),1))
GPR2['lidarSD'] = lidarSD

GPR2.head()
#len(GPRLiDAR)

In [ ]:
dataFusion.head()

Data specific for date analysis only

In [ ]:
##Filtering by date for analysis on dates -- 
GPRLiDARdate1 = GPR1.sort_values(by ='test', ascending = True)
GPRLiDARdate1 = GPR1.groupby(['test', 'date']).agg({'twt': 'median', 'easting': 'first', 'northing': 'first', 'lidarSD': 'first'})

GPRLiDARdate2 = GPR2.sort_values(by ='test', ascending = True)
GPRLiDARdate2 = GPR2.groupby(['test', 'date']).agg({'twt': 'median', 'easting': 'first', 'northing': 'first', 'lidarSD': 'first'})
length = len(GPRLiDARdate2)
print(length)

Data we have grouped fully based on lidar cell!

In [ ]:
##Filtering ignoring different dates. 
GPRLiDAR1 = GPR1.sort_values(by ='test', ascending = True)
GPRLiDAR1 = GPR1.groupby(['test']).agg({'twt': 'median', 'easting': 'first', 'northing': 'first', 'lidarSD': 'first', 'date': 'mean'})

GPRLiDAR2 = GPR2.sort_values(by ='test', ascending = True)
GPRLiDAR2 = GPR2.groupby(['test']).agg({'twt': 'median', 'easting': 'first', 'northing': 'first', 'lidarSD': 'first', 'date': 'mean'})
length = len(GPRLiDAR2)
print(length)

In [ ]:
GPRLiDAR1.head()

## Calculate snow density 
- eq 1. hs = TWT(v)/2  hs = Lidar snow depth, TWT = gpr two way travel time, V = wave speed
- eq 2. V = C/ sqrt(e),  V = wave speed, C = speed of light (0.3m/ns), e = dielectric permittivity
- eq 3. e = (1+0.845d)^2  Kovacs et al. 1995 d = density (kg/m3 /1000)

re-arrange eq 1.
- V = hs(2)/TWT

re-arrange eq 2.
- e = C^2/V^2

re-arrange eq 3.
- d = (sqrt(e)-1)/0.845

- update lidarSD column name 

In [ ]:
#First calculation 1
#V = wave speed
GPRLiDAR1['v']= (GPRLiDAR1["lidarSD"]*2)/GPRLiDAR1["twt"]
GPRLiDAR2['v']= (GPRLiDAR2["lidarSD"]*2)/GPRLiDAR2["twt"]

## date specific data set
GPRLiDARdate2['v']= (GPRLiDARdate2["lidarSD"]*2)/GPRLiDARdate2["twt"]
GPRLiDARdate1['v']= (GPRLiDARdate1["lidarSD"]*2)/GPRLiDARdate1["twt"]

In [ ]:
#second Calculation 
#e = dielectric permitivity
GPRLiDAR1['e'] = 0.3**2/(GPRLiDAR1['v']**2)
GPRLiDAR2['e'] = 0.3**2/(GPRLiDAR2['v']**2)

## date specific data set
GPRLiDARdate2['e'] = 0.3**2/(GPRLiDARdate2['v']**2)
GPRLiDARdate1['e'] = 0.3**2/(GPRLiDARdate1['v']**2)

In [ ]:
#third calculation
# density
# d = (sqrt(e)-1)/0.845
GPRLiDAR1['dens'] = (((GPRLiDAR1['e'])**(1/2))-1)/0.845
#convert units to kg/m^3
GPRLiDAR1['dens'] = GPRLiDAR1['dens']*1000

GPRLiDAR2['dens'] = (((GPRLiDAR2['e'])**(1/2))-1)/0.845
#convert units to kg/m^3
GPRLiDAR2['dens'] = GPRLiDAR2['dens']*1000

##date specific dataset
GPRLiDARdate1['dens'] = (((GPRLiDARdate1['e'])**(1/2))-1)/0.845
#convert units to kg/m^3
GPRLiDARdate1['dens'] = GPRLiDARdate1['dens']*1000

GPRLiDARdate2['dens'] = (((GPRLiDARdate2['e'])**(1/2))-1)/0.845
#convert units to kg/m^3
GPRLiDARdate2['dens'] = GPRLiDARdate2['dens']*1000

In [ ]:
GPRLiDAR1.head()

## Simple Median Filter to remove 
- running median filter using a buffer


In [ ]:
import scipy.spatial as spatial
tmp = GPRLiDAR1[0:1000000:100]


In [ ]:
#points = np.array((GPRLiDAR1["easting"],GPRLiDAR1["northing"]))
points = np.array((tmp["easting"],tmp["northing"]))
points = points.transpose()
print(points)
point_tree = spatial.cKDTree(points)
buffer = 25
out = point_tree.query_ball_point(points, 25)

In [ ]:
denseout = []
for x in out:
    denseout.append(tmp.iloc[x]['dens'].median())
len(denseout)

## Calculate SWE
- GPR density (kg/m3)
- LiDAR Snow Depth (cm)

- SWE (cm) = LiDAR SD (cm) * GPR Density (g/cm3)

In [ ]:
## all dates of same location between two time frames data 
GPRLiDAR1['SWE'] = GPRLiDAR1['lidarSD'] * (GPRLiDAR1['dens']/1000)
GPRLiDAR2['SWE'] = GPRLiDAR2['lidarSD'] * (GPRLiDAR2['dens']/1000)

##dates only filtered by
GPRLiDARdate1['SWE'] = GPRLiDARdate1['lidarSD'] * (GPRLiDARdate1['dens']/1000)
GPRLiDARdate2['SWE'] = GPRLiDARdate2['lidarSD'] * (GPRLiDARdate2['dens']/1000)

GPRLiDARdate1.head()

## Export CSV with new values
Table should include:
- ASO X 
- ASO Y
- ASO snow depth
- ASO SWE
- GPR TWT
- GPR Density
- GPR/LiDAR SWE



In [ ]:
## GPRLiDAR1 = January 28th - February 2nd ( Flight February 2nd 2020)
GPRLiDAR1.to_csv('GPRLiDAR1.csv')
## GPRLiDAR2 = Febraury 4 - February 9 (Flight February 13th 2020
GPRLiDAR2.to_csv('GPRLiDAR2.csv')

##filtered by dates and lidar cell location
GPRLiDARdate1.to_csv('GPRLiDARdate1.csv')
GPRLiDARdate2.to_csv('GPRLiDARdate2.csv')
